# Great Lakes Models Dashboard
Visualizing ocean model data using the [pyviz](pyviz.org) tools. 

In [ ]:
import xarray as xr

In [ ]:
%%time
#url = 'https://opendap.co-ops.nos.noaa.gov/thredds/dodsC/LOOFS/fmrc/Aggregated_7_day_LOOFS_Fields_Forecast_best.ncd'
url = 'http://opendap.co-ops.nos.noaa.gov/thredds/dodsC/NOAA/LOOFS/MODELS/201903/glofs.loofs.fields.forecast.20190320.t06z.nc'
ds = xr.open_dataset(url)

Find all the data variables that depend on time (and are not time `bounds`)

In [ ]:
ds['temp']

In [ ]:
ds = ds.set_coords(names=['lon','lat'])

In [ ]:
rho_vars = []
for var in ds.data_vars:
    if len(ds[var].dims) > 0:
        if 'time' in ds[var].dims[0] and (not 'validtime' in var) and ('nx' in ds[var].dims[-1]):
            rho_vars.append(var)

In [ ]:
rho_vars

Import the [pyviz](http://pyviz.org) tools we need

In [ ]:
from cartopy import crs as ccrs
import hvplot.xarray
import holoviews as hv
from geoviews import tile_sources as gvts
import panel as pn
from holoviews import streams
import numpy as np

Create widget for variable selection

In [ ]:
var_select = pn.widgets.Select(name='Model Variables:', options=rho_vars, 
                               value='temp')

Create widget for basemap selection

In [ ]:
base_map_select = pn.widgets.Select(name='Basemap:', options=gvts.tile_sources, value=gvts.OSM)                                                    

The `plot` function below creates the `hvplot` panel layout object.  We specify a basemap, pick the `quadmesh` plot type for the selected variable, and indicate we want to `rasterize` the plot so that we can render massive meshes in the browser. We also specify the `groupby` parameter as the list of dimensions that remains after we remove Y and X: `ds[var].dims[:-2]`, which automatically handles variables with either dimensions [T, Y, X] or [T, Z, Y, X].  We also specify which `bokeh` controls we want to be active by default:  the `wheel_zoom` and `pan` controls.

We also change the default slider to a selection widget for the `time` dimension so that specific times are easy to select.  See https://stackoverflow.com/a/54912917/2005869

In [ ]:
crs = ccrs.PlateCarree()

Create a stream to get location of tap

In [ ]:
tap_mesh = streams.Tap(x = 1,y = 1)

In [ ]:
def plot(var=None, base_map=None):
    base_map = base_map or base_map_select.value
    var = var or var_select.value
    mesh = ds[var][-24:,:,:].hvplot.quadmesh(x='lon', y='lat', rasterize=True, title=var,
                                    width=600, height=400, crs=crs,
                                    groupby=list(ds[var].dims[:-2]), cmap='jet')
    
    tap_mesh.source = mesh # Provide source to the tap stream
    
    overlay = (base_map * mesh.opts(alpha=0.7)).opts(active_tools=['wheel_zoom', 'pan'])
    widgets = {dim: pn.widgets.Select for dim in ds[var].dims[:-2]}
    return pn.pane.HoloViews(overlay, widgets=widgets).layout

In [ ]:
def on_var_select(event):
    var = event.obj.value
    dashboard[-1] = plot(var=var)

In [ ]:
def on_base_map_select(event):
    base_map = event.obj.value
    dashboard[-1] = plot(base_map=base_map)

The following function converts `lat lon` to `nx ny` for selection of data points from dataset.

In [ ]:
def create_time_series(x,y):
    sel_lat = y 
    sel_lon = x 

    a = abs(ds.lat-sel_lat)+abs(ds.lon-sel_lon)
    i,j = np.unravel_index(a.argmin(),a.shape)

    time_series = ds.sel(nx =j,ny = i,sigma = 0.0)
    return time_series[var_select.value].hvplot()

In [ ]:
var_select.param.watch(on_var_select, parameter_names=['value']);
base_map_select.param.watch(on_base_map_select, parameter_names=['value']);

A dynamic map for time series plotting

In [ ]:
ts_map = hv.DynamicMap(lambda x,y: create_time_series(x,y), streams=[tap_mesh])

In [ ]:
dashboard = pn.Column(var_select, base_map_select, plot(var_select.value))

In [ ]:
dashboard

In [ ]:
var_select.value = 'air_v'

In [ ]:
ts_map 

Tap anywhere on the mesh to get a time series map